Import data from API

In [96]:
import re
import json
import requests
import urllib.request
from base64 import b64encode
import pandas as pd
import numpy as np
from pandas.io.json import json_normalize
import operator, functools
import matplotlib

In [97]:
r = requests.get("http://strainapi.evanbusse.com/FsMDSqK/strains/search/all")
tex=(r.text)

In [99]:
data= json.loads(tex)
data= pd.DataFrame(data)
data= data.transpose()
efectos= pd.json_normalize(data.effects)
efectos
data=data.reset_index()
data=pd.concat([data, efectos], axis=1)
data=data.drop('effects', axis=1)


Create lambda to find effects

In [125]:
flat = data.positive.apply(lambda x: 'Relaxed'in x)
df1 = data[flat]


Find positive effects funcion

In [126]:
def find_positive(data, lst):
    indices= set(data.index)
    for values in lst:
        flat = data.positive.apply(lambda x: values in x)
        indices=indices.intersection(data[flat].index)
    
    return data.iloc[list(indices)]

Find medical issue to solve - function

In [127]:
def find_medical(data, lst):
    indices= set(data.index)
    for values in lst:
        flat = data.medical.apply(lambda x: values in x)
        indices=indices.intersection(data[flat].index)
    
    return data.iloc[list(indices)]

Avoide negative effects to solve - function

In [128]:
def avoid(data, lst):
    indices= set(data.index)
    for values in lst:
        flat = data.negative.apply(lambda x: values not in x)
        indices=indices.intersection(data[flat].index)
    
    return data.iloc[list(indices)]

In [129]:
def find_master(data, positive=[], negative=[], medical=[]):
    indices= set(data.index)
    indices= indices.intersection(find_positive(data, positive).index)
    indices= indices.intersection(find_medical(data, medical).index)
    indices= indices.intersection(avoid(data, negative).index)
    return data.iloc[list(indices)]

Define effects 

In [145]:
all_effects = {"effect":"Relaxed","type":"positive"},{"effect":"Dizzy","type":"negative"},{"effect":"Hungry","type":"positive"},{"effect":"Euphoric","type":"positive"},{"effect":"Happy","type":"positive"},{"effect":"Depression","type":"medical"},{"effect":"Insomnia","type":"medical"},{"effect":"Pain","type":"medical"},{"effect":"Stress","type":"medical"},{"effect":"Cramps","type":"medical"},{"effect":"Creative","type":"positive"},{"effect":"Energetic","type":"positive"},{"effect":"Talkative","type":"positive"},{"effect":"Lack of Appetite","type":"medical"},{"effect":"Nausea","type":"medical"},{"effect":"Dry Mouth","type":"negative"},{"effect":"Headache","type":"medical"},{"effect":"Uplifted","type":"positive"},{"effect":"Tingly","type":"positive"},{"effect":"Paranoid","type":"negative"},{"effect":"Sleepy","type":"positive"},{"effect":"Fatigue","type":"medical"},{"effect":"Headaches","type":"medical"},{"effect":"Dry Eyes","type":"negative"},{"effect":"Focused","type":"positive"},{"effect":"Eye Pressure","type":"medical"},{"effect":"Anxious","type":"negative"},{"effect":"Giggly","type":"positive"},{"effect":"Aroused","type":"positive"},{"effect":"Inflammation","type":"medical"},{"effect":"Spasticity","type":"medical"},{"effect":"Seizures","type":"medical"},{"effect":"Muscle Spasms","type":"medical"}
all_eff= pd.DataFrame.from_dict(all_effects) 

In [131]:
medical_issues= all_eff.loc[all_eff['type'] == "medical"]
medicals=medical_issues[['effect']]

In [132]:
positive_effects= all_eff.loc[all_eff['type'] == "positive"]
positives=positive_effects[['effect']]

In [133]:
negative_effects= all_eff.loc[all_eff['type'] == "negative"]
negatives=negative_effects[['effect']]

LET'S START

CHOOSE HOW YOU WANT TO FEEL

In [134]:
print(positives)

       effect
0     Relaxed
2      Hungry
3    Euphoric
4       Happy
10   Creative
11  Energetic
12  Talkative
17   Uplifted
18     Tingly
20     Sleepy
24    Focused
27     Giggly
28    Aroused


In [135]:
print("hit enter after each effect. q to quit")
positive=[]
while True:
    p = input()
    if str(p) == 'q':
        break
    positive.append(p)

hit enter after each effect. q to quit
Aroused
q


SELECT MEDICAL SITUATION YOU NEED TO FIX

In [136]:
print(medicals)

              effect
5         Depression
6           Insomnia
7               Pain
8             Stress
9             Cramps
13  Lack of Appetite
14            Nausea
16          Headache
21           Fatigue
22         Headaches
25      Eye Pressure
29      Inflammation
30        Spasticity
31          Seizures
32     Muscle Spasms


In [152]:
print("hit enter after each effect. q to quit")
medical=[]
while True:
    m = input()
    if str(m) == 'q':
        break
    medical.append(m)

hit enter after each effect. q to quit
Depression
Pain
q


SELECT THE NEGATIVE EFFECTS YOU WANT TO AVOID

In [154]:
print(negatives)

       effect
1       Dizzy
15  Dry Mouth
19   Paranoid
23   Dry Eyes
26    Anxious


In [156]:
print("hit enter after each effect. q to quit")
negative=[]
while True:
    n = input()
    if str(n) == 'q':
        break
    negative.append(n)

hit enter after each effect. q to quit
Dizzy
q


FIND FUNCTION

In [219]:
recommended_strains=find_master(data, positive, negative, medical)

Your cannabis race probability

In [221]:
#extra: your most compatible cannabis race
print('Your Cannabis race probablity')
races= find_master(data, positive, negative, medical)[['race']]
recommended_race = races['race'].value_counts().to_dict()
s = sum(recommended_race.values())
for k, v in recommended_race.items():
    pct = v * 100.0 / s
    print(k, pct)

Your Cannabis race probablity
hybrid 45.0
indica 32.5
sativa 22.5


In [217]:
#find available flavors
flavors=recommended_strains[['index', 'flavors', 'race']].explode('flavors')
flav=flavors[['flavors']]
flav=flav.to_numpy()
set(map(tuple, flav))


{('Ammonia',),
 ('Apricot',),
 ('Berry',),
 ('Blueberry',),
 ('Butter',),
 ('Cheese',),
 ('Chemical',),
 ('Citrus',),
 ('Coffee',),
 ('Diesel',),
 ('Earthy',),
 ('Flowery',),
 ('Grape',),
 ('Lavender',),
 ('Lemon',),
 ('Lime',),
 ('Menthol',),
 ('Orange',),
 ('Pear',),
 ('Pepper',),
 ('Pine',),
 ('Pineapple',),
 ('Plum',),
 ('Pungent',),
 ('Sage',),
 ('Skunk',),
 ('Spicy/Herbal',),
 ('Strawberry',),
 ('Sweet',),
 ('Tar',),
 ('Tree Fruit',),
 ('Tropical',),
 ('Woody',)}

In [225]:
final_recommendation = flavors[flavors['flavors'] == "Pineapple"]
final_recommendation

,index,flavors,race
1402,Pineapple Punch,Pineapple,sativa
876,Hawaiian Cookies,Pineapple,hybrid
